In [3]:
import pandas as pd
import re
from bs4 import BeautifulSoup
#from sentence_transformers import SentenceTransformer

# Entities Exploration

## Concept

In [9]:
concept_df = pd.read_json("../data/MOOCCube/entities/concept.json",lines=True)

In [10]:
concept_df.sample(5)

,id,name,en,explanation
90902,K_政策本位主义_管理科学技术,政策本位主义,policy parochialism,学科：管理科学技术_公共管理_公共政策 定义：国家、政府机关及其内部单位，从本身的立场或利益...
73715,K_机械合金化粉_材料科学技术,机械合金化粉,mechanically alloyed powder,学科：材料科学技术_金属材料 _粉末冶金材料 定义：几种单质粉末，经过高能球磨，粉末反复发生...
96119,K_电气与机械工程合同条件_管理科学技术,电气与机械工程合同条件,condition of contract for electrical and mecha...,学科：管理科学技术_项目管理与工程管理_采购与合同管理 定义：适用于由承包商按照雇主要求，设...
105204,K_实效伴流_船舶工程,实效伴流,effective wake,学科：船舶工程_船舶性能及其试验 定义：根据船模自航试验和螺旋桨敞水试验的结果在同转速、等推...
21469,K_输尿管镜检查术_医学,输尿管镜检查术,ureteroscopy,学科：医学_泌尿外科学 见载：《医学名词 第六分册》第一版


## Course

In [7]:
course_df = pd.read_json("../data/MOOCCube/entities/course.json",lines=True)

In [65]:
course_df.sample(5)

,id,name,prerequisites,about,core_id,video_order,display_name,chapter
697,C_course-v1:TCIS+2019032101X+2019_T2,中华文化要义：大一统（2019夏）,1.苏力：《大国宪制：历史中国的制度构成》，北京大学出版社2017年版。2.施展：《枢纽：3...,<p>\r\n\t&nbsp;&nbsp;十九大报告凸显文化自信的重要意义，强调引导人们树立...,C_course-v1:TCIS+2019032101X+2019_T2,"[V_4fb8b8d233724a7a978d61544f1cb88d, V_a2c56aa...","[前言, 1.1殷周天命观的剧变, 1.2宗法制与封建制, 1.3天下体系与礼乐文明, 2....","[01.01.01.01, 02.01.01.01, 02.02.01.01, 02.03...."
638,C_course-v1:SCUT+20180427001+2019_T1,微积分1（Calculus I）（2019春）,数学、英语等,<p>\r\n\t<span> </span> \r\n</p>\r\n<p>\r\n\t微...,C_course-v1:SCUT+20180427001+2019_T1,"[V_523f20de0c4644b8a480aec07b57bef2, V_c1498d8...","[Course Introduction, Rigorous Study of Limits...","[01.01.01.01, 02.02.01.01, 02.03.01.01, 02.04...."
94,C_course-v1:RiceX+AdvENVSCI_3x+sp,AP*备考——环境科学，第三部分：污染和资源（自主模式）,无,<p>备考AP环境科学考试需要深入了解有关环境科学的相关话题，同时还需熟悉AP考试的题目类型...,C_course-v1:RiceX+AdvENVSCI_1x+sp,"[V_f8cb9f425b9b45949c2604bc96fef3da, V_d4ca16c...","[Introduction, Introduction to Pollution, Lect...","[01.01.01.01, 02.02.01.01, 02.03.02.01, 02.05...."
240,C_course-v1:JNU+2018012501X+sp,民法与生活,无,<p>\r\n\t民法是从日常生活中抽象出来的法律规则，是民事领域的基本法律规范，是社会生活...,C_course-v1:JNU+2018012501X+sp,"[V_0490f5aaf1ae40aa99be57a8c887271c, V_505da76...","[Video, Video, Video, Video, Video, Video, Vid...","[01.01.01.01, 02.01.01.01, 02.02.01.01, 03.01...."
197,C_course-v1:XJTU+C00204+sp,中国哲学经典著作导读（自主模式）,无先修要求,<p>\n\t<br />\n</p>\n<p>\n\t课程内容：<span></span>...,C_course-v1:XJTU+C00204+sp,"[V_xjtu-C00204-1-1-1, V_xjtu-C00204-1-1-2, V_x...","[燕连福第1节：《周易》的作者1, 燕连福第1节：《周易》的作者2, 燕连福第1节：《周易》...","[01.01.01.01, 01.01.02.01, 01.01.03.01, 01.02...."


In [66]:
def remove_html_and_carriage_returns(text):
    soup = BeautifulSoup(text, "html.parser")
    text_without_html = soup.get_text().replace("\n","").replace("\r","").replace("\t","")
    
    return text_without_html

course_df["about"] = course_df["about"].apply(remove_html_and_carriage_returns)
course_df.sample(5)

,id,name,prerequisites,about,core_id,video_order,display_name,chapter
465,C_course-v1:TsinghuaX+60250101X+2019_T1,英文科技论文写作与学术报告（2019春）,无,发表高质量的学术论文和参与国际学术交流已经成为科学研究必不可少的环节及衡量学术成就的重要指标...,C_course-v1:TsinghuaX+60250101X+2019_T1,"[V_87abc33eb27f4a0b87b1f1ea6732fbff, V_7101bd8...","[Video, Video, Video, Video, Video, Video, Vid...","[01.01.01.01, 02.01.01.01, 02.02.01.01, 02.03...."
295,C_course-v1:SWPU+3615001035+2019_T1,钻井与完井工程（2019春）,《石油工程测井》《工程流体力学》《石油工程岩石力学》,《钻井与完井工程》是西南石油大学石油工程专业本科生核心主干课程，课堂教学内容主要包括井身结构...,C_course-v1:SWPU+3615001035+2019_T1,"[V_dacb5b9acb474b21b2dbc3f090a11150, V_142fe6d...","[1.1我国石油工业概述, 1.2钻井的目的与井的分类, 1.3钻井完井技术发展概述, 2....","[01.01.01.01, 01.02.01.01, 01.03.01.01, 02.01...."
317,C_course-v1:HNU+20180529001+2019_T1,设计的力量（2019春）,艺术简史、科技前沿,本课程主要以人类设计历史，尤其是过去250年中发生的重大历史事件为线索，梳理设计在国家发展过...,C_course-v1:HNU+20180529001+2019_T1,"[V_6a502c95b6eb415e9265e5ab1bf084ec, V_1af5423...","[本章简介, 01生存设计, 02中国的手工艺设计, 03西方设计之源, 04中世纪与哥特式...","[01.01.01.01, 01.02.01.01, 01.03.01.01, 01.04...."
657,C_course-v1:BIFT+1301990078+2019_T1,服装数字科技（2019春）,计算机应用基础,《服装数字科技》这门课通过介绍服装计算机辅助设计，虚拟服装，服装人体数字化，数字化服装定制，...,C_course-v1:BIFT+1301990078+2019_T1,"[V_166f5e9255cf40d2999a0ff498e52287, V_39b5d75...","[Video, Video, Video, Video, Video, Video, Vid...","[01.01.01.01, 01.02.01.01, 01.03.01.01, 01.04...."
650,C_course-v1:KMUSTX+1803168+2019_T1,基因工程（2019春）,掌握分子生物学和微生物学基本知识,基因工程是生物相关学科专业（生物工程、生物技术、生物科学等）重要的专业课之一，是生命科学的重...,C_course-v1:KMUSTX+1803168+2018_T2,"[V_9f3cc65bd68447b3b489be6ba8510a39, V_db660a2...","[Video, Video, Video, Video, Video, Video, Vid...","[01.01.01.02, 01.02.01.02, 01.03.01.01, 01.03...."


In [69]:
# sentences = []
# course_ids = [] 
# for index, row in course_df.iterrows():
#     course_ids.append(row["id"])
#     sentences.append(row["about"])
# model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')
# embeddings = model.encode(sentences)


In [ ]:
course_df = course_df.explode('video_order').reset_index()
course_df.head(5)

,index,id,name,prerequisites,about,core_id,video_order,display_name,chapter
0,0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_f6f710068b994452885b90e11b6ee5c5,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
1,0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_73395684a8e24e2ea378b926214335ea,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
2,0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_8bc91d3fc5914379ae31723cdc0de9f1,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
3,0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_8aef0ca48efc445c9edf545114ba22a9,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
4,0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_ae9b4e594d4347a4b3da95dc1851eac3,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."


In [ ]:
course_df = course_df.drop('index', axis=1)
course_df.head(5)

,id,name,prerequisites,about,core_id,video_order,display_name,chapter
0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_f6f710068b994452885b90e11b6ee5c5,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
1,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_73395684a8e24e2ea378b926214335ea,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
2,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_8bc91d3fc5914379ae31723cdc0de9f1,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
3,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_8aef0ca48efc445c9edf545114ba22a9,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
4,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_ae9b4e594d4347a4b3da95dc1851eac3,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."


## Papers

In [11]:
paper_df = pd.read_json("../data/MOOCCube/entities/paper.json",lines=True)

In [12]:
paper_df.sample(5)

,id,name,abstract,authors,doi,lang,num_citation,pdf,sourcetype,title,venue,year,pages,urls
626316,P_5c757d22f56def9798aacfb4,,We use an integral equation formulation approa...,"[{'name': 'Roland Mallier'}, {'name': 'Joanna ...",10.1017/S1446181118000160,en,0,NaN,publication,Integral Equation Formulation For Shout Options,"{'info': {'name': 'Anziam Journal'}, 'issue': ...",2018,"{'end': '85', 'start': '65'}",NaN
255526,P_53e9b29cb7602d9703d4269e,,"Through the example of our daily life,the mode...","[{'id': '53f46ad6dabfaeee22a6430b', 'name': 'W...",,en,2,NaN,publication,Two-fold Compound Generalized Poisson Process,{'info': {'name': 'Journal of Changshu Institu...,2007,"{'end': '30', 'start': '27'}",NaN
398759,P_53e9bb1db7602d970475a500,,We study vector fields on a disk satisfying tw...,"[{'id': '53f43671dabfaee0d9b6571c', 'name': 'D...",10.1063/1.531021,en,83,NaN,publication,Vector Fields on a Disk with Mixed Boundary Co...,{'info': {'name': 'Journal of Mathematical Phy...,1994,"{'end': '', 'start': ''}",[http://dx.doi.org/10.1063/1.531021]
142054,P_53e9a1fab7602d9702afbde7,,,"[{'id': '53f45670dabfaee2a1d742d2', 'name': 'S...",10.1021/ja9831002,en,54,NaN,publication,Steric Control of Electron Transfer. Changeove...,{'info': {'name': 'Journal of The American Che...,1999,"{'end': '626', 'start': '617'}",[http://dx.doi.org/10.1021/ja9831002]
96842,P_55a52b2424012c2a39131d04,,,"[{'id': '560c47b045ce1e5960624a36', 'name': 'M...",,en,10,,publication,Decreased carotid flow with ascending aortic c...,"{'info': {'name': 'Circulation'}, 'issue': 'Su...",1972,"{'end': '3', 'start': 'I130'}",[http://www.ncbi.nlm.nih.gov/pubmed/5025049?re...


## School

In [13]:
school_df = pd.read_json("../data/MOOCCube/entities/school.json",lines=True)

In [14]:
school_df.sample(5)

,id,name,about
63,S_CUC,中国传媒大学,中国传媒大学是教育部直属的国家“211工程”、“985工程”优势学科创新平台，是国家首批“双...
176,S_UIR,国际关系学院,国际关系学院（University of International Relations），...
87,S_LZU,兰州大学,兰州大学是中华人民共和国教育部直属全国重点大学，中央直管副部级建制，位列国家首批“双一流(A...
136,S_SDUx,山东大学,山东大学（Shandong University），简称山大（SDU），由中华人民共和国教育...
80,S_CDUTCM,成都中医药大学,成都中医药大学（Chengdu University of TCM），位于四川省会成都市，是...


## Teacher

In [15]:
teacher_df = pd.read_json("../data/MOOCCube/entities/teacher.json",lines=True)

In [16]:
teacher_df.sample(5)

,id,name,about
846,T_袁佐,袁佐,袁佐 副教授，八十年代初就读中央美术学院，后毕业于美国麻省艺术学院 (Massachuset...
1090,T_Cath Lovelock,Cath Lovelock,Cath Lovelock 是海洋植物学家，专攻生态学和沿海植物群落的生态生理学。她的研究小...
77,T_李小芬,李小芬,副教授，体育教育训练学博士，北京体育大学艺术学院体育舞蹈教研室主任；中国体育舞蹈联合会会国际...
314,T_侯春菊,侯春菊,侯春菊，博士，副教授
578,T_刘昭伟,刘昭伟,博士，副教授，博士生导师，2012年入选教育部“新世纪人才支持计划”。2008年至2009年...


## User

In [17]:
user_df = pd.read_json("../data/MOOCCube/entities/user.json",lines=True)

In [18]:
user_df.sample(5)

,id,name,course_order,enroll_time
137026,U_1853485,戈傲,"[C_course-v1:TsinghuaX+20220053X_2015_T2+sp, C...","[2018-01-09 14:50:06, 2017-08-15 22:46:47]"
50864,U_7808691,闻人乐家,"[C_course-v1:TsinghuaX+00670122X+sp, C_course-...","[2017-10-13 11:22:56, 2018-03-04 14:47:59]"
139572,U_9422877,楼醉薇,"[C_course-v1:TsinghuaX+20330334X_2015_2+sp, C_...","[2019-01-24 14:37:34, 2018-07-03 17:07:33]"
85644,U_1075631,玉永思,"[C_course-v1:TsinghuaX+00612642X+sp, C_course-...","[2015-09-13 20:20:07, 2015-09-13 20:40:12]"
79825,U_7129960,庆凝梦,"[C_course-v1:TsinghuaX+00740123_X+sp, C_course...","[2019-02-20 16:56:05, 2019-02-20 16:56:17]"


## Video

In [19]:
video_df = pd.read_json("../data/MOOCCube/entities/video.json",lines=True)

In [20]:
video_df.sample(5)

,id,name,start,end,text
22252,V_9d1985379b0e43f2b72bf8bb0ce7e561,问题解答视频 (2:16),"[0, 4004, 12012, 20020, 25025, 33033, 38038, 4...","[4000, 12004, 20012, 25020, 33025, 38033, 4003...","[我说过控制器方法中的一个实例方法，, 它的值能保留多长时间？读一读然后思考。初次投票。, ..."
5376,V_7ece9c6eb78145dfa1a1bfc686423f40,梵文藏文转写的作用与意义,"[46360, 49440, 54120, 57760, 60720, 65160, 672...","[49440, 54120, 57760, 60720, 65160, 67200, 690...",[དེས་ན་ད་ལྟ་བོད་ཡིག་དང་མཉམ་དུ་སྤྱོད་ཀྱིན་ཡོད་པ...
29017,V_fcf4d120d526436ebccf2291ea8bc2e5,15.5 新古典主义时期金属工艺,"[11520, 14480, 19040, 20080, 22400, 25480, 319...","[14440, 18480, 20040, 22360, 24880, 31880, 349...","[新古典主义时期的金属工艺, 尤其是贵金属工艺有了新的发展, 这个时候, 新古典主义时期的金..."
29991,V_68fcba30bd9141d4bf679c82229f6bc0,母函数小结,"[352, 1311, 3471, 4911, 7591, 10951, 13831, 17...","[1271, 3431, 4871, 7551, 10911, 13791, 17191, ...","[那么这节课呢, 我们给大家介绍了一个新的概念, 也就是母函数, 母函数的样子和函数是一模一..."
11586,V_8275fd7c582a4ed4aa54b50b0ca97558,着色问题的等价类,"[13030, 15770, 18560, 21390, 22700, 24310, 262...","[15470, 18170, 21220, 22240, 24240, 26180, 271...","[我们刚才一直都在讲置换群, 而置换群的对象无非就是一些文字, 那么再回到我们本节课一开始,..."
